# CORE TEAM
1. NUR HUDA RIYANTONI (09020620037)
2. ZUYYINA HAWANI (09040620070)

In [1]:
import urllib.request
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
from datetime import datetime

Get data realtime

In [2]:
url = 'https://bmkg-content-inatews.storage.googleapis.com/live30event.xml'
resp = urllib.request.urlopen(url)
xml_data = resp.read()
root = ET.fromstring(xml_data)

Create Data List

In [3]:
event_list =[]
status_list =[]
waktu_list = []
lintang_list = []
bujur_list = []
dalam_list =[]
mag_list =[]
fokal_list =[]
area_list = []

Print data realtime 

In [ ]:
# Take first 'Gempa' element
gempa = root.find('gempa')

# Take all 'person' element
for gempa in root.findall('gempa'):
    event = gempa.find('eventid').text
    status = gempa.find('status').text
    waktu = gempa.find('waktu').text
    # waktus= datetime.strptime(waktu, "%Y/%m/%d %H:%M:%S.%f")
    lintang = gempa.find('lintang').text
    bujur = gempa.find('bujur').text
    dalam = gempa.find('dalam').text
    mag = gempa.find('mag').text
    fokal = gempa.find('fokal').text
    area = gempa.find('area').text
    event_list.append(event) 
    status_list.append(status)
    waktu_list.append(waktu)
    lintang_list.append(lintang)
    bujur_list.append(bujur)
    dalam_list.append(dalam)
    mag_list.append(mag) 
    fokal_list.append(fokal)
    area_list.append(area)

Show off data list to data frame

In [5]:
import pandas as pd

In [6]:
#Input data list to dataframe
event_list
status_list
waktu_list 
lintang_list 
bujur_list 
dalam_list
mag_list
fokal_list
area_list 

datas = pd.DataFrame({'ID Event':event_list, 'Status Gempa':status_list, 'Waktu Gempa':waktu_list, 'Lintang Gempa':lintang_list,
                    'Bujur Gempa':bujur_list, 'Kedalaman Gempa':dalam_list, 'Magnitudo Gempa':mag_list, 'Fokal Gempa':fokal_list,
                    'Area Gempa':area_list,})
datas

,ID Event,Status Gempa,Waktu Gempa,Lintang Gempa,Bujur Gempa,Kedalaman Gempa,Magnitudo Gempa,Fokal Gempa,Area Gempa
0,bmg2023lwcj,confirmed,2023/06/18 15:05:00.079,-23.78,-175.92,75,5.8,undetermined,Tonga Islands Region
1,bmg2023lwch,confirmed,2023/06/18 15:02:44.915,-8.70,119.32,103,3.5,undetermined,"Flores Region, Indonesia"
2,bmg2023lvwh,confirmed,2023/06/18 12:00:55.095,-3.34,130.92,41,3.4,undetermined,"Seram, Indonesia"
3,bmg2023lvun,confirmed,2023/06/18 11:06:38.156,-9.21,123.86,34,3.1,undetermined,Timor Region
4,bmg2023lvsh,confirmed,2023/06/18 09:59:30.500,-15.42,167.53,130,5.3,undetermined,Vanuatu Islands
...,...,...,...,...,...,...,...,...,...
179,bmg2023llny,confirmed,2023/06/12 20:26:22.053,-8.73,110.78,31,2.8,undetermined,"Java, Indonesia"
180,bmg2023lllf,confirmed,2023/06/12 19:03:28.560,1.49,127.19,127,2.5,undetermined,"Halmahera, Indonesia"
181,bmg2023lllc,confirmed,2023/06/12 18:59:37.018,-8.75,119.60,115,3.2,undetermined,"Flores Region, Indonesia"
182,bmg2023llkt,confirmed,2023/06/12 18:49:48.308,-9.33,123.99,43,3.1,undetermined,Timor Region


# INPUT DATA TO DATABASE

In [7]:
import mysql.connector

Create Database

In [14]:
# db = mysql.connector.connect(
#     host = "localhost",
#     user = "root",
#     passwd = "",
# )

# cursor = db.cursor()
# cursor.execute("CREATE DATABASE gempa")

# print("Database gempa berhasil dibuat")

Connect with database that was created before

In [ ]:
db = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd="",
  database ="gempa"
)

if db.is_connected():
  print("Success connect to database gempa")
else:
  print("turn on your XAMPP!")

Create Table

In [16]:
# db = mysql.connector.connect(
#     host = "localhost",
#     user = "root",
#     passwd = "",
#     database = "gempa"
# )

# cursor = db.cursor()

# table = """CREATE TABLE gempafix(
#     id_event VARCHAR(20),
#     status VARCHAR(10),
#     waktu TIMESTAMP PRIMARY KEY,
#     lintang DOUBLE,
#     garis_bujur DOUBLE,
#     kedalaman DOUBLE,
#     magnitudo DOUBLE,
#     fokal VARCHAR (15), 
#     area VARCHAR (100)
#     )"""

# cursor.execute(table)

Input data to database

In [12]:
cursor = db.cursor()
datas
nama_tabel = "info"

for i, row in datas.iterrows():
    id_event = row['ID Event']
    status = row['Status Gempa']
    waktu = row['Waktu Gempa']
    lintang = row['Lintang Gempa']
    garis_bujur = row['Bujur Gempa']
    kedalaman = row['Kedalaman Gempa']
    magnitudo = row['Magnitudo Gempa']
    fokal = row['Fokal Gempa']
    area = row['Area Gempa']

    sql = "INSERT INTO {} (id_Event, status, waktu, lintang, garis_bujur, kedalaman, magnitudo, fokal, area) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)".format(nama_tabel)
    val = (id_event, status, waktu, lintang, garis_bujur, kedalaman, magnitudo, fokal, area)
    cursor.execute(sql, val)
    db.commit()
cursor.close()


True

Try input 1 data, when data was updated on web

In [17]:
# cursor = db.cursor()
# nama_tabel = "info"   

# sql = "INSERT INTO {} (id_Event, status, waktu, lintang, garis_bujur, kedalaman, magnitudo, fokal, area) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)".format(nama_tabel)
# val = (event_list[0], status_list[0], waktu_list[0], lintang_list[0],bujur_list[0], dalam_list[0], mag_list[0], fokal_list[0], area_list[0])
# cursor.execute(sql, val)
# db.commit()

Cek result on database

In [13]:
cursor = db.cursor()
sql = "SELECT * FROM info"
cursor.execute(sql)

results = cursor.fetchall()
for data in results:
  print(data)

('bmg2023lwcj', 'confirmed', datetime.datetime(2023, 6, 18, 15, 5), -23.78, -175.92, 75.0, 5.8, 'undetermined', 'Tonga Islands Region')
('bmg2023lwch', 'confirmed', datetime.datetime(2023, 6, 18, 15, 2, 44), -8.7, 119.32, 103.0, 3.5, 'undetermined', 'Flores Region, Indonesia')
('bmg2023lvwh', 'confirmed', datetime.datetime(2023, 6, 18, 12, 0, 55), -3.34, 130.92, 41.0, 3.4, 'undetermined', 'Seram, Indonesia')
('bmg2023lvun', 'confirmed', datetime.datetime(2023, 6, 18, 11, 6, 38), -9.21, 123.86, 34.0, 3.1, 'undetermined', 'Timor Region')
('bmg2023lvsh', 'confirmed', datetime.datetime(2023, 6, 18, 9, 59, 30), -15.42, 167.53, 130.0, 5.3, 'undetermined', 'Vanuatu Islands')
('bmg2023lvsa', 'confirmed', datetime.datetime(2023, 6, 18, 9, 50, 59), -8.05, 114.16, 10.0, 2.2, 'undetermined', 'Bali Region, Indonesia')
('bmg2023lvrw', 'confirmed', datetime.datetime(2023, 6, 18, 9, 46, 18), -8.56, 118.94, 156.0, 4.0, 'undetermined', 'Sumbawa Region, Indonesia')
('bmg2023lvru', 'confirmed', datetime.d